# MINLP

In [3]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from gekko import GEKKO
from tabulate import tabulate

In [41]:
# Main function to solve the MINLP pricing optimization problem
def Solve_MINLP(df, WarmStart_delta, WarmStart_d, TotalBudget, RevenueTarget, MarginTarget,
                       MaxDiscount, MinDiscount, MaxSKU, MaxBundles, MaxCons, w3, w4):
    # Initialize GEKKO model with local (offline) solving
    m = GEKKO(remote=False)
    m.options.SOLVER = 1  # Use APOPT solver (good for MINLP problems)
 
    # Compute weights for revenue and margin to be used in objective function
    total_target = RevenueTarget + MarginTarget
    w1 = RevenueTarget / total_target
    w2 = MarginTarget / total_target
 
    # w1 = 5000
    # w2 = 5000
    # Initialize constants
    BigM = 1e6
    n_items = len(df)  # Number of SKUs
 
    # Declare decision variables:
    # Binary selection variable δ_i (1 if selected, 0 otherwise)
    delta = [m.Var(value=WarmStart_delta[i], lb=0, ub=1, integer=True) for i in range(n_items)]
    # Discount variable d_i (fractional discount between 0 and 1)
    d = [m.Var(value=WarmStart_d[i], lb=0, ub=1) for i in range(n_items)]
    # Revenue and margin variables (continuous, ≥ 0)
    r = [m.Var(lb=0) for _ in range(n_items)]
    m_i = [m.Var(lb=0) for _ in range(n_items)]
 
    # Slack variables for deviation from revenue/margin targets
    z_R = m.Var(lb=0)
    z_M = m.Var(lb=0)
 
    # Extract pricing, sales, elasticity, and cost data from dataframe
    P0 = df['Regular_Price'].values
    S0 = df['Total_Sales'].values
    PE = df['Price_Elasticity'].values
    cost = df['unit_cost'].values
 
    # Define constraints and intermediate equations per SKU

    for i in range(n_items):
        # Discounted price formula using binary and discount variables
        P = P0[i] * (1 - delta[i] * d[i])
        # Demand function using own-price elasticity (log-log form)
        S = S0[i] * ((P / P0[i]) ** PE[i])
        # Revenue = Price × Demand
        m.Equation(r[i] == S * P)
        # Margin = Demand × (Price - Cost)
        m.Equation(m_i[i] == S * (P - cost[i]))
 
        # Constraint: Discounted price must fall within allowed min and max bounds
        m.Equation(P >= P0[i] * (1 - MaxDiscount[i]))
        m.Equation(P <= P0[i] * (1 - MinDiscount[i]))


    # Constraints
    m.Equation(m.sum([delta[i] for i in range(n_items)]) <= MaxSKU)
    # m.Equation(m.sum(m_i) >= MarginTarget - z_M)
    # m.Equation(m.sum(r) >= RevenueTarget - z_R)

    m.Equation(m.sum(m_i) >= m.max2(TotalBudget, MarginTarget - z_M))
    #m.Equation(m.sum(m_i) >= TotalBudget)
    # Objective


    # Demand increase term: S/S0 - 1 (percentage demand increase)
    DemandIncrease = m.sum([(S0[i] * ((P0[i] * (1 - delta[i] * d[i])) / P0[i])**PE[i] - S0[i]) for i in range(n_items)])
    obj = w1 * m.sum(r) + w2 * m.sum(m_i) + w5 * DemandIncrease - w3 * z_R - w4 * z_M

    
    #obj = w1 * m.sum(r) + w2 * m.sum(m_i) - w3 * z_R - w4 * z_M
    m.Obj(-obj)
    try:
        m.solve(disp=True)
    except Exception as e:
        print("⚠️ GEKKO failed. Opening model folder for diagnosis...")
        m.open_folder()
        raise e
    return {
        'delta': [d.value[0] for d in delta],
        'd': [dd.value[0] for dd in d],
        'r': [rr.value[0] for rr in r],
        'margin': [mm.value[0] for mm in m_i],
        'z_R': z_R.value[0],
        'z_M': z_M.value[0]
    }


def print_results(results, df):
    delta = results['delta']
    discounts = results['d']
    revenues = results['r']
    margins = results['margin']
    product_names = df['Product_Description'].values
    summary_df = pd.DataFrame({'Product Name ': product_names,'Selected ': ['Selected' if int(d) == 1 else 'Not Selected' for d in delta],'Discount % ': [f"{round(dis * 100, 2)}%" if sel >= 0.5 else "0.0%" for dis, sel in zip(discounts, delta)],'Revenue ': [f"${round(r, 2)}" for r in revenues],'Margin ': [f"${round(m, 2)}" for m in margins]})
    print("\n OPTIMIZATION RESULTS: ITEM-LEVEL OVERVIEW")
    print(tabulate(summary_df, headers='keys', tablefmt='fancy_grid', showindex=False))
    print("\n\033[1;32m Summary Metrics:\033[0m")
    print(f" Total Selected Items: {sum(1 for d in delta if d >= 0.5)}")
    print(f" Total Revenue: \033[1m${round(sum(revenues), 2)}\033[0m")
    print(f" Total Margin: \033[1m${round(sum(margins), 2)}\033[0m")
    # print(f" Revenue Gap (z_R):{results['z_R']:.2f}")
    # print(f" Margin Gap (z_M):{results['z_M']:.2f}")
   

if __name__ == "__main__":
    # Load data (make sure this file exists and has the correct columns)
    df = pd.read_csv('Subset1_BS_S001_loyalonly (1).csv')
 
    # Initialize model inputs
    n = len(df)
    WarmStart_delta = [0] * n           # Start with no items selected
    WarmStart_d = [0.0] * n             # Start with 0% discount across items
    MaxDiscount = [0.35] * n             # Max discount = 30%
    MinDiscount = [0.0] * n            # Min discount = 0%
    MaxSKU = 12                         # Max SKUs allowed to discount
    MaxBundles = 0                      # Bundle logic not implemented yet
    MaxCons = 0                         # Constraint groups not implemented yet
    RevenueTarget = 20e3
    MarginTarget = 10e3
    TotalBudget = 10e3
 
 
    
    # Budget in terms of allowable margin reduction
    #RevenueTarget = 10000              # Minimum revenue desired
    #MarginTarget = 3000                # Minimum margin desired
    w3 = 10000                          # Penalty for not hitting revenue target
    w4 = 10000                           # Penalty for not hitting margin target
    w5 = 10000
    # Run optimization and print results
    results = Solve_MINLP(df, WarmStart_delta, WarmStart_d, TotalBudget, RevenueTarget,
                                  MarginTarget, MaxDiscount, MinDiscount, MaxSKU, MaxBundles,
                                  MaxCons, w3, w4)
    print_results(results, df)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            6
   Constants    :            0
   Variables    :          152
   Intermediates:            0
   Connections  :          108
   Equations    :          105
   Residuals    :          105
 
 Number of state variables:            154
 Number of total equations: -          111
 Number of slack variables: -           42
 ---------------------------------------
 Degrees of freedom       :              1
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -4.72E+06 Gap:       NaN
Iter:     2 I: -1 Tm:      0.04 NLPi:   30 Dpth:    1 Lvs:    2 Obj: -4.72E

## MILP

In [ ]:
from gekko import GEKKO
import numpy as np

def Solve_MILP_Phase2(
    Phase1_Selection_delta_star,  # Binary selection vector from Phase 1
    Phase1_Discounts_d_star,      # Discount percentages from Phase 1
    DemandForecast_S0,            # Base demand forecast for each item
    PriceElasticity_PE,           # Price elasticity for each item
    DemandSubstitution_S_matrix,  # Substitution effect matrix
    P_reg,                        # Regular prices
    C,                            # Unit costs
    P0,                           # Reference prices
    Targets,                      # Revenue and margin targets
    TotalBudget,                  # Overall budget constraint
    Weights,                      # Weights for revenue and margin in objective
    PenaltyWeights,              # Weights for penalty variables
    BusinessRules,                # Pricing bounds and business constraints
    SolverOptions                 # Options for Gekko solver
):
    """
    Solve the Phase 2 MILP: Linear Perturbation & Demand Substitution
    """

    n = len(P_reg)  # Number of items

    # --- Step 3.1: Compute optimized prices from Phase 1 ---
    P_star = P_reg * (1 - Phase1_Discounts_d_star)

    # --- Step 3.2: Compute demand S*_i using only price elasticity (no substitution yet) ---
    S_star = np.zeros(n)
    for i in range(n):
        if Phase1_Selection_delta_star[i] == 1:
            S_star[i] = DemandForecast_S0[i] * (P_star[i] / P0[i]) ** PriceElasticity_PE[i]
        else:
            S_star[i] = 0

    # --- Step 3.3: Compute derivatives of demand w.r.t. price at P*_i ---
    D = np.zeros(n)
    for i in range(n):
        if Phase1_Selection_delta_star[i] == 1:
            D[i] = PriceElasticity_PE[i] * (S_star[i] / P_star[i])
        else:
            D[i] = 0

    # --- Initialize MILP Model ---
    m = GEKKO(remote=False)
    m.options.SOLVER = 1  # APOPT MILP solver
    m.options.IMODE = 3   # Steady state optimization

    # --- Decision Variables: ΔP_i (Price perturbations), only for selected items ---
    DeltaP = [m.Var(value=0, lb=BusinessRules['MinPriceChange'][i], ub=BusinessRules['MaxPriceChange'][i]) 
              if Phase1_Selection_delta_star[i]==1 else m.Param(0) for i in range(n)]

    # --- Penalty variables for unmet revenue and margin targets ---
    z_R = m.Var(lb=0)
    z_M = m.Var(lb=0)

    # --- Step 3.4: Linearized demand with substitution effects ---
    S2 = [m.Intermediate(
            S_star[i] +
            D[i] * DeltaP[i] +
            m.sum([DemandSubstitution_S_matrix[i][j] * D[j] * DeltaP[j] for j in range(n) 
                   if (j != i and Phase1_Selection_delta_star[j] == 1)])
        ) if Phase1_Selection_delta_star[i]==1 else m.Param(0) for i in range(n)]

    # --- Step 3.4: Compute linearized revenue and margin ---
    r2 = [m.Intermediate(
            P_star[i]*S_star[i] + 
            P_star[i]*(D[i]*DeltaP[i] + m.sum([DemandSubstitution_S_matrix[i][j]*D[j]*DeltaP[j] 
                                               for j in range(n) if (j != i and Phase1_Selection_delta_star[j]==1)])) + 
            DeltaP[i]*S_star[i]
        ) if Phase1_Selection_delta_star[i]==1 else m.Param(0) for i in range(n)]

    m2 = [m.Intermediate(
            (P_star[i] - C[i])*S_star[i] +
            (P_star[i] - C[i])*(D[i]*DeltaP[i] + m.sum([DemandSubstitution_S_matrix[i][j]*D[j]*DeltaP[j] 
                                                       for j in range(n) if (j != i and Phase1_Selection_delta_star[j]==1)])) +
            DeltaP[i]*S_star[i]
        ) if Phase1_Selection_delta_star[i]==1 else m.Param(0) for i in range(n)]

    # --- Step 3.5: Define Objective Function ---
    w1 = Weights['Revenue']
    w2 = Weights['Margin']
    w3 = PenaltyWeights['RevenuePenalty']
    w4 = PenaltyWeights['MarginPenalty']

    # Maximize revenue and margin while penalizing deviations from targets
    obj = w1 * m.sum(r2) + w2 * m.sum(m2) - w3 * z_R - w4 * z_M
    m.Obj(-obj)  # GEKKO minimizes, so we use negative

    # --- Constraints: Ensure revenue and margin meet targets (with slack) ---
    m.Equation(m.sum(r2) + z_R >= Targets['RevenueTarget'])
    m.Equation(m.sum(m2) + z_M >= Targets['MarginTarget'])

    # --- Budget Constraint: Ensure adjusted prices respect the total budget ---
    BudgetUsed = m.sum([(P_star[i] + DeltaP[i]) * S2[i] for i in range(n) if Phase1_Selection_delta_star[i]==1])
    m.Equation(BudgetUsed <= TotalBudget)

    # --- Solve MILP ---
    m.solve(disp=SolverOptions.get('disp', True))

    # --- Step 3.6: Extract optimal price perturbations ---
    OptimalPerturbations_DeltaP = np.array([
        DeltaP[i].value[0] if Phase1_Selection_delta_star[i]==1 else 0 for i in range(n)
    ])

    # --- Step 3.7: Compute Final Prices and Discounts ---
    FinalPrices = {}
    FinalDiscounts = {}
    Final_Selection_delta = Phase1_Selection_delta_star.copy()

    for i in range(n):
        if Final_Selection_delta[i] == 1:
            price = P_star[i] + OptimalPerturbations_DeltaP[i]
            # Clamp price to allowed discount range
            MinP = P_reg[i] * (1 - BusinessRules['MaxDiscount'][i])
            MaxP = P_reg[i] * (1 - BusinessRules['MinDiscount'][i])
            price = min(max(price, MinP), MaxP)
            FinalPrices[i] = price
            FinalDiscounts[i] = 1 - price / P_reg[i]
        else:
            FinalPrices[i] = P_reg[i]
            FinalDiscounts[i] = 0

    # --- Final Key Performance Indicators (KPIs) ---
    Final_KPIs = {
        'Final_Revenue': sum([FinalPrices[i]*S2[i].value[0] for i in range(n) if Final_Selection_delta[i]==1]),
        'Final_Margin': sum([(FinalPrices[i]-C[i])*S2[i].value[0] for i in range(n) if Final_Selection_delta[i]==1]),
        'Final_Sales': sum([S2[i].value[0] for i in range(n) if Final_Selection_delta[i]==1]),
        'Final_BudgetUsed': BudgetUsed.value[0]
    }

    return Final_Selection_delta, FinalDiscounts, Final_KPIs


In [ ]:
import pandas as pd

def print_product_KPIs(Final_Selection_delta, FinalDiscounts, FinalPrices, S2, C, P_reg):
   """
    Prints a tabular summary of key KPIs for each selected product.

    Parameters:
    - Final_Selection_delta: binary list indicating selected products (1 if selected)
    - FinalDiscounts: dictionary of final discount values for each product
    - FinalPrices: dictionary of final price values for each product
    - S2: list of GEKKO Intermediates (expected sales after optimization)
    - C: list of cost values for each product
    - P_reg: list of original (regular) prices for each product
    """
    data = []

    for i in range(len(Final_Selection_delta)):
        if Final_Selection_delta[i] == 1:
            final_price = FinalPrices[i]
            final_discount = FinalDiscounts[i]
            final_sales = S2[i].value[0]
            final_revenue = final_price * final_sales
            final_margin = (final_price - C[i]) * final_sales
            original_price = P_reg[i]

            data.append({
                'Product ID': i,
                'Original Price': round(original_price, 2),
                'Final Price': round(final_price, 2),
                'Discount (%)': round(final_discount * 100, 2),
                'Expected Sales': round(final_sales, 2),  # Extract predicted sales from GEKKO variable
                'Expected Revenue': round(final_revenue, 2),
                'Expected Margin': round(final_margin, 2)
            })

    df = pd.DataFrame(data)
    print("\nKey Performance Indicators for Selected Products:\n")
    print(df.to_string(index=False))
    
    return Final_Selection_delta, FinalDiscounts, FinalPrices, S2, Final_KPIs



Final_Selection_delta, FinalDiscounts, Final_KPIs = Solve_MILP_Phase2(...)
print_product_KPIs(Final_Selection_delta, FinalDiscounts, FinalPrices, S2, C, P_reg)